In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split

import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.utils import np_utils

import h5py

X_train = np.load("20170827_mawilab_flow_000features.pkl")
Y_train = np.load("20170827_mawilab_flow_000labels.pkl")

scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)
scaler = Normalizer().fit(Y_train)
Y_train = scaler.transform(Y_train)

# Test Set 1 : 20170827_mawilab_flow_001
X_test1 = np.load("20170827_mawilab_flow_001features.pkl")
Y_test1 = np.load("20170827_mawilab_flow_001labels.pkl")
scaler = Normalizer().fit(X_test1)
X_test1 = scaler.transform(X_test1)
scaler = Normalizer().fit(Y_test1)
Y_test1 = scaler.transform(Y_test1)

# Test Set 2 : 20170827_mawilab_flow_002
X_test2 = np.load("20170827_mawilab_flow_002features.pkl")
Y_test2 = np.load("20170827_mawilab_flow_002labels.pkl")
scaler = Normalizer().fit(X_test2)
X_test2 = scaler.transform(X_test2)
scaler = Normalizer().fit(Y_test2)
Y_test2 = scaler.transform(Y_test2)

# Test Set 3 : 20170827_mawilab_flow_003
X_test3 = np.load("20170827_mawilab_flow_003features.pkl")
Y_test3 = np.load("20170827_mawilab_flow_003labels.pkl")
scaler = Normalizer().fit(X_test3)
X_test3 = scaler.transform(X_test3)
scaler = Normalizer().fit(Y_test3)
Y_test3 = scaler.transform(Y_test3)

# Test Set 4: 20170827_mawilab_flow_004
X_test4 = np.load("20170827_mawilab_flow_004features.pkl")
Y_test4 = np.load("20170827_mawilab_flow_004labels.pkl")
scaler = Normalizer().fit(X_test4)
X_test4 = scaler.transform(X_test4)
scaler = Normalizer().fit(Y_test4)
Y_test4 = scaler.transform(Y_test4)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test1 = np.reshape(X_test1, (X_test1.shape[0],X_test1.shape[1],1))
X_test2 = np.reshape(X_test2, (X_test2.shape[0],X_test2.shape[1],1))
X_test3 = np.reshape(X_test3, (X_test3.shape[0],X_test3.shape[1],1))
X_test4 = np.reshape(X_test4, (X_test4.shape[0],X_test4.shape[1],1))


C:\Anaconda\envs\GPU\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Anaconda\envs\GPU\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda\envs\GPU\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [2]:
import numpy as np
import pickle
import pandas as pd 
X_test_ = pd.read_pickle("./dataset/kdd_test__2labels.pkl").as_matrix()
Y_test_ = pd.read_pickle("./dataset/kdd_test__2labels_y.pkl").as_matrix()

X_train_ = pd.read_pickle("./dataset/kdd_train__2labels.pkl").as_matrix()
Y_train_ = pd.read_pickle("./dataset/kdd_train__2labels_y.pkl").as_matrix()

print(X_test_.shape, Y_test_.shape)
print(X_train_.shape, Y_train_.shape)

X_train_ = np.reshape(X_train_, (X_train_.shape[0],X_train_.shape[1],1))
X_test_ = np.reshape(X_test_, (X_test_.shape[0],X_test_.shape[1],1))


(11850, 124) (11850,)
(25192, 124) (25192,)


In [ ]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(124, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

In [ ]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
cnn1d_1.fit(X_train_, Y_train_, batch_size=64, validation_data=(X_test_, Y_test_) ,epochs=10)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("20170827_mawilab_flow_000.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("20170827_mawilab_flow_000.h5")
print("Saved model to disk")

Train on 25192 samples, validate on 11850 samples
Epoch 1/10
